# Homework 4 (ADM) - Movie Recommendation System

Author: Viktoriia Vlasenko  

Email: [vlasenko.2088928@studenti.uniroma1.it](mailto:vlasenko.2088928@studenti.uniroma1.it)

***

## 1. Recommendation System with LSH

In this section, you will implement a recommendation system by identifying users with similar preferences and suggesting movies based on their behavior. 
Specifically, you will implement your version of the [**LSH algorithm**](https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/).

### 1.1 Data Preparation

Download the MovieLens dataset from [here](https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset?select=rating.csv). After downloading, explore the dataset to understand the structure and identify any preprocessing steps needed.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import time
from tqdm import tqdm
import asyncio
from tqdm.asyncio import tqdm
from itertools import combinations
from sklearn.metrics import mean_absolute_error, precision_recall_fscore_support
from collections import defaultdict

**Load Data**

In [4]:
movie = pd.read_csv("/kaggle/input/movielens-20m-dataset/movie.csv")
movie.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [5]:
rating = pd.read_csv("/kaggle/input/movielens-20m-dataset/rating.csv")
rating.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

### 1.2 Minhash Signatures

Using the `userId` and `movieId` columns, implement your own MinHash function. This function will hash each user's watched movie list, creating a representation that allows for quick comparisons of user similarities.

- **Important:** Implement your MinHash function from scratch—**do not use any pre-built hash functions.**
- Use your MinHash function to generate signature vectors for each user based on their rated movies.
- Experiment with different hash functions and threshold values to find the most effective configurations. Report these results.
- Read the class materials and, if necessary, conduct an internet search. The description of hash functions in the [book](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf) may be helpful as a reference.

In [30]:
user_movie_map = rating.groupby('userId')['movieId'].apply(set)
user_movie_map

userId
1         {2, 3081, 4105, 1036, 5146, 541, 29, 32, 4128,...
2         {3, 260, 2948, 2951, 1544, 1673, 266, 908, 245...
3         {512, 1, 3072, 2053, 2054, 2567, 1544, 2571, 1...
4         {6, 519, 520, 10, 19, 531, 32, 420, 165, 548, ...
5         {2, 515, 260, 1028, 648, 11, 140, 141, 780, 10...
                                ...                        
138489    {2571, 2959, 912, 2324, 2455, 1562, 29, 1061, ...
138490    {1543, 3079, 2573, 2575, 17, 529, 531, 1041, 2...
138491    {1, 8961, 33794, 3608, 4128, 2722, 2085, 2857,...
138492    {2054, 2571, 21, 2599, 5673, 1073, 6711, 58, 2...
138493    {1, 2, 2050, 2051, 69644, 2065, 18, 19, 2078, ...
Name: movieId, Length: 138493, dtype: object

`generate_diverse_hash_functions` is genereting different types of hash functions for creating the signatures. I picked 10 types of different function and randomly generated the coefficients for them.

Since I am using random approach for creatiung hash function there is no need in further shuffling the hash functions.

In [25]:
def generate_diverse_hash_functions(num_hashes=10, max_value=1000):
    """
    Generate diverse hash functions for MinHash experimentation.
    """
    hash_functions = []
    for i in range(num_hashes):
        # Generate random coefficients for the hash functions
        a = random.randint(1, max_value)
        b = random.randint(0, max_value)
        c = random.randint(0, max_value)
        p = max_value + 1  # Use a large prime

        # Linear hash function
        hash_functions.append(lambda x, a=a, b=b, p=p: (a * x + b) % p)
        
        # Quadratic hash function
        hash_functions.append(lambda x, a=a, b=b, c=c, p=p: (a * x**2 + b * x + c) % p)
        
        # Cubic hash function
        hash_functions.append(lambda x, a=a, b=b, c=c, p=p: (a * x**3 + b * x**2 + c) % p)
        
        # Modulo-based hash function
        hash_functions.append(lambda x, a=a, b=b, p=p: ((x + a) * (x + b)) % p)
        
        # XOR-based hash function
        hash_functions.append(lambda x, a=a, p=p: (a ^ x) % p)
        
        # Composite hash function
        hash_functions.append(lambda x, a=a, b=b, p=p: ((a * x) + b) // 2 % p)
        
        # Logarithmic-based hash function
        hash_functions.append(lambda x, a=a, p=p: int((a * (1 + x).bit_length()) % p))
        
        # Exponential-based hash function
        hash_functions.append(lambda x, a=a, p=p: int((a * (2 ** x)) % p))
        
        # Prime shift hash function
        hash_functions.append(lambda x, a=a, p=p: (a * (x + p)) % p)
        
        # Bit-shift hash function
        hash_functions.append(lambda x, a=a, p=p: ((x << a) + (x >> a)) % p)
    
    # Limit to 10 unique hash functions
    return hash_functions[:num_hashes]

I will generate 10 different hash function cause bigger amaount of function is computationaly expensive for my set up (Kaggle notebook).

In [26]:
diverse_hash_functions = generate_diverse_hash_functions(num_hashes=10, max_value=1000)

For the further experiment with configuration I will compare different types of functions and only linear ones.

In [6]:
linear_hash_functions = []
for a, b, p in [
    (3, 7, 101),   # h1(x) = (3x + 7) % 101
    (5, 11, 103),  # h2(x) = (5x + 11) % 103
    (7, 13, 107),  # h3(x) = (7x + 13) % 107
    (11, 17, 109), # h4(x) = (11x + 17) % 109
    (13, 19, 113), # h5(x) = (13x + 19) % 113
    (17, 23, 127), # h6(x) = (17x + 23) % 127
    (19, 29, 131), # h7(x) = (19x + 29) % 131
    (23, 31, 137), # h8(x) = (23x + 31) % 137
    (29, 37, 139), # h9(x) = (29x + 37) % 139
    (31, 41, 149)  # h10(x) = (31x + 41) % 149
]:
    linear_hash_functions.append(lambda x, a=a, b=b, p=p: (a * x + b) % p)

In [28]:
# Async function for computing a user's signature
async def compute_user_signature(user_id, movies, hash_functions):
    """Compute the MinHash signature for a single user asynchronously."""
    signature = []
    for hash_fn in hash_functions:
        min_hash = float('inf')
        for movie in movies:
            hash_value = hash_fn(movie)
            min_hash = min(min_hash, hash_value)
        signature.append(min_hash)
    return user_id, signature

async def generate_user_signatures(user_movie_map, hash_functions, batch_size=1000):
    """
    Generate MinHash signatures for all users using the given hash functions.
    
    Args:
        user_movie_map: Dictionary mapping user IDs to sets of movie IDs.
        hash_functions: List of (a, b, p) tuples defining the hash functions.
        batch_size: Number of users to process in each batch.
    
    Returns:
        Dictionary of user signatures.
    """
    all_user_signatures = {}
    user_ids = list(user_movie_map.keys())
    
    print("Generating MinHash signatures for users...")
    start_time = time.time()
    
    # Async signature computation with batched processing
    with tqdm(total=len(user_ids), desc="User Signatures", unit="user") as pbar:
        for i in range(0, len(user_ids), batch_size):
            batch_ids = user_ids[i:i + batch_size]
            
            # Create async tasks for batch processing
            tasks = [
                compute_user_signature(user_id, user_movie_map[user_id], hash_functions)
                for user_id in batch_ids
            ]
            
            # Process the batch
            batch_results = await asyncio.gather(*tasks)
            
            # Collect signatures for each user in the batch
            for user_id, signature in batch_results:
                all_user_signatures[user_id] = signature
            
            # Update progress bar
            pbar.update(len(batch_ids))
    
    print(f"All user signatures computed in {time.time() - start_time:.2f} seconds.")
    return all_user_signatures

I generated user signatures using onlu linear hash functions and separately 10 different types of hash functions.

In [31]:
# Generate user signatures
linear_user_signatures = await generate_user_signatures(user_movie_map, linear_hash_functions)
diverse_user_signatures = await generate_user_signatures(user_movie_map, diverse_hash_functions)

Generating MinHash signatures for users...


User Signatures: 100%|██████████| 138493/138493 [57:32<00:00, 40.11user/s]

All user signatures computed in 3452.44 seconds.


`save_user_signatures_to_files` and `load_user_signatures_from_files` are created for saving runtime `user_signatures` into the files and uploading if there is a need.

In [9]:
async def save_user_signatures_to_files(user_signatures, output_dir, selector='linear'):
    filename = f"{output_dir}/user_signatures_{selector}.json"
    with open(filename, "w") as f:
        json.dump(user_signatures, f, indent=4)
    print(f"User signatures saved to {filename}")

In [6]:
# Load user signatures from the file
def load_user_signatures_from_files(output_dir, selector='linear'):
    user_signatures = {}
    filename = f"{output_dir}/user_signatures_{selector}.json"
    with open(filename, "r") as f:
        user_signatures = json.load(f)
    print(f"User signatures from loaded from {filename}")
    # Cast values to integer values: when dumping a dictionary into json values are written as strings
    user_signatures = {int(user_id): list(map(int, signature)) for user_id, signature in user_signatures.items()}
    return user_signatures

In [7]:
output_dir = '/kaggle/working'

Selector in files writing/reading functions `selector='linear'` is for distinguish different `user_signatures` for an experiment. In 1.3 will be used only one set of `user_signatures` that will show better metrics in the experiment. 

In [11]:
await save_user_signatures_to_files(linear_user_signatures, output_dir)
await save_user_signatures_to_files(diverse_user_signatures, output_dir, 'diverse')

User signatures saved to /kaggle/working/user_signatures_linear.json
User signatures saved to /kaggle/working/user_signatures_diverse.json


In [7]:
linear_user_signatures = load_user_signatures_from_files(output_dir)
diverse_user_signatures = load_user_signatures_from_files(output_dir, 'diverse')

User signatures from loaded from /kaggle/working/user_signatures_diverse.json


Here I'm comparing the signatures pairwise and evaluting metrics.

In [15]:
# Compute true Jaccard similarity
def compute_true_jaccard(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

# Compute estimated similarity from MinHash signatures
def compute_minhash_similarity(signature1, signature2):
    matches = sum(1 for h1, h2 in zip(signature1, signature2) if h1 == h2)
    return matches / len(signature1)

async def compare_user_pair(user1, user2, user_signatures, threshold):
    """
    Compare a pair of users and compute metrics asynchronously.
    """
    # Compute True Jaccard similarity
    true_jaccard = compute_true_jaccard(user_movie_map[user1], user_movie_map[user2])
    
    # Compute MinHash Estimated Jaccard Similarity
    estimated_jaccard = compute_minhash_similarity(user_signatures[user1], user_signatures[user2])
    
    # Check threshold conditions
    true_positive = estimated_jaccard >= threshold and true_jaccard >= threshold
    false_positive = estimated_jaccard >= threshold and true_jaccard < threshold
    false_negative = estimated_jaccard < threshold and true_jaccard >= threshold

    # Log comparison results
    return {
        "true_jaccard": true_jaccard,
        "estimated_jaccard": estimated_jaccard,
        "true_positive": true_positive,
        "false_positive": false_positive,
        "false_negative": false_negative,
    }

async def evaluate_all_pairs_async(user_signatures, thresholds, batch_size=1000):
    """
    Evaluate MinHash signatures for all user pairs asynchronously for multiple thresholds.
    """
    results = {}
    user_ids = list(user_signatures.keys())
    total_users = len(user_ids)
    total_pairs = total_users * (total_users - 1) // 2

    print(f"Total users: {total_users}, Total pairs: {total_pairs}")

    for threshold in thresholds:
        print(f"Evaluating for threshold: {threshold}...")

        # Create tasks for all user pairs
        tasks = []
        for user1, user2 in combinations(user_ids, 2):
            tasks.append(compare_user_pair(user1, user2, user_signatures, threshold))

        # Process tasks in batches with progress tracking
        results_list = []
        with tqdm(total=len(tasks), desc=f"Processing pairs for threshold {threshold}", unit="pair") as pbar:
            for i in range(0, len(tasks), batch_size):
                task_batch = tasks[i:i + batch_size]
                batch_results = await asyncio.gather(*task_batch)
                results_list.extend(batch_results)
                pbar.update(len(task_batch))  # Update progress bar

        # Aggregate results
        true_positives = sum(res["true_positive"] for res in results_list)
        false_positives = sum(res["false_positive"] for res in results_list)
        false_negatives = sum(res["false_negative"] for res in results_list)
        all_true_similarities = [res["true_jaccard"] for res in results_list]
        all_estimated_similarities = [res["estimated_jaccard"] for res in results_list]

        # Compute metrics
        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1 = (2 * precision * recall / (precision + recall)) if precision + recall > 0 else 0
        mae = mean_absolute_error(all_true_similarities, all_estimated_similarities)

        print(f"Completed evaluation for threshold {threshold}: precision: {precision}, recall: {recall}, f1_score: {f1}, mae: {mae}")

In [16]:
# Define thresholds
thresholds = [0.3, 0.5, 0.7]

To short off the process of comparing all signatures pairs I decided to pick a subset and run an experiment on it. The subset will be picked randomly according to the desired `max_pairs` value.

In [17]:
# Select a subset of user_signatures with approximately max_pairs
def get_subset_user_signatures(user_signatures, max_pairs=200000):
    user_ids = list(user_signatures.keys())
    total_users = len(user_ids)
    
    # Calculate how many users are required for max_pairs
    subset_size = 1
    while subset_size * (subset_size - 1) // 2 <= max_pairs:
        subset_size += 1
    subset_size -= 1  # Step back to ensure we don't exceed max_pairs

    print(f"Selecting {subset_size} users for approximately {max_pairs} pairs.")
    
    # Randomly sample user IDs
    sampled_user_ids = random.sample(user_ids, subset_size)
    
    # Create a subset of user_signatures
    subset_signatures = {user_id: user_signatures[user_id] for user_id in sampled_user_ids}
    return subset_signatures

In [20]:
# Get a subset of user_signatures
subset_linear_user_signatures = get_subset_user_signatures(linear_user_signatures, max_pairs=2000000)
subset_diverse_user_signatures = get_subset_user_signatures(diverse_user_signatures, max_pairs=2000000)

Selecting 2000 users for approximately 2000000 pairs.
Selecting 2000 users for approximately 2000000 pairs.


In [21]:
# Run async evaluation with tracing and progress tracking
await evaluate_all_pairs_async(subset_linear_user_signatures, thresholds)

Total users: 2000, Total pairs: 1999000
Evaluating for threshold: 0.3...


Processing pairs for threshold 0.3: 100%|██████████| 1999000/1999000 [01:18<00:00, 25553.53pair/s]


Completed evaluation for threshold 0.3: precision: 0.010454081041510865, recall: 0.9215413184772516, f1_score: 0.020673637727045488, mae: 0.2936536385124049
Evaluating for threshold: 0.5...


Processing pairs for threshold 0.5: 100%|██████████| 1999000/1999000 [01:15<00:00, 26479.18pair/s]


Completed evaluation for threshold 0.5: precision: 0.0015205543718719693, recall: 0.8118811881188119, f1_score: 0.003035423765517216, mae: 0.2936536385124049
Evaluating for threshold: 0.7...


Processing pairs for threshold 0.7: 100%|██████████| 1999000/1999000 [01:15<00:00, 26509.06pair/s]


Completed evaluation for threshold 0.7: precision: 0.00023634058827320972, recall: 0.8870967741935484, f1_score: 0.0004725552782276599, mae: 0.2936536385124049


{}

In [22]:
# Run async evaluation with tracing and progress tracking
await evaluate_all_pairs_async(subset_diverse_user_signatures, thresholds)

Total users: 2000, Total pairs: 1999000
Evaluating for threshold: 0.3...


Processing pairs for threshold 0.3: 100%|██████████| 1999000/1999000 [01:14<00:00, 26993.33pair/s]


Completed evaluation for threshold 0.3: precision: 0.028887808970796524, recall: 0.8313908313908314, f1_score: 0.05583553604723742, mae: 0.13670679434116426
Evaluating for threshold: 0.5...


Processing pairs for threshold 0.5: 100%|██████████| 1999000/1999000 [01:13<00:00, 27038.91pair/s]


Completed evaluation for threshold 0.5: precision: 0.010281556469471686, recall: 0.7367044463818657, f1_score: 0.02028008112032448, mae: 0.13670679434116426
Evaluating for threshold: 0.7...


Processing pairs for threshold 0.7: 100%|██████████| 1999000/1999000 [01:14<00:00, 26984.07pair/s]


Completed evaluation for threshold 0.7: precision: 0.0018585921164717726, recall: 0.75, f1_score: 0.0037079953650057938, mae: 0.13670679434116426


{}

**Linear Hash Functions**
| Metric        | Threshold (0.3) | Threshold (0.5) | Threshold (0.7) |
|---------------|-----------------|-----------------|-----------------|
| **Precision** | 0.01045         | 0.00153         | 0.00026         |
| **Recall**    | 0.92154         | 0.81181         | 0.88709         |
| **F1 Score**  | 0.02067         | 0.00305         | 0.00047         |
| **MAE**       | 0.29365         | 0.29365         | 0.29365         |

**Different Hash Functions**
| Metric        | Threshold (0.3) | Threshold (0.5) | Threshold (0.7) |
|---------------|-----------------|-----------------|-----------------|
| **Precision** | 0.02888         | 0.01028         | 0.00185         |
| **Recall**    | 0.83139         | 0.73670         | 0.75000         |
| **F1 Score**  | 0.05583         | 0.02020         | 0.00370         |
| **MAE**       | 0.13670         | 0.13670         | 0.13670         |

**Observations:**
1. **Precision**: Different hash functions show significantly higher precision than linear hash functions across all thresholds. This suggests that different hash functions are better at reducing false positives, as they capture more unique and independent aspects of the data.
3. **Recall**: Linear hash functions achieve higher recall, especially at lower thresholds (e.g., 0.3). This indicates that linear hash functions are better at identifying similar user pairs but might introduce more false positives.
4. **F1 Score**: The F1 score for diverse hash functions is consistently higher, particularly at threshold 0.3, where the improvement is most pronounced. This balance between precision and recall highlights the practical effectiveness of different hash functions.
5. **MAE**: The MAE for different hash functions is significantly lower than for linear hash functions. This demonstrates that the estimated Jaccard similarity from different hash functions is much closer to the true Jaccard similarity, improving accuracy.

**Conclusions**:

For 1.3 I will use different types of hash functions cause they demonstrated better accuracy that only linear functions.

### 1.3 Locality-Sensitive Hashing (LSH)

Now that you have generated MinHash user signatures, apply Locality-Sensitive Hashing (LSH) to cluster similar users.

1. **Bucket Creation:** For each user, divide the MinHash signature into bands and hash each band to form buckets. Users with similar bands should fall into the same buckets.
   - **Debugging Tip:** After creating buckets, check a few bucket contents to verify that multiple users are being grouped in the same buckets.
   
2. **Query:** For a given user, identify the **two most similar users** based on their bucket placement. If a user doesn’t have any similar users in their bucket, adjust the parameters until similar users are found.

3. **Movie Recommendation Logic:**
   - If both similar users have rated a movie, recommend this movie based on the **average rating**.
   - If there are no commonly rated movies, recommend the top-rated movies of the most similar user.

4. **Final Recommendation:** Provide **at most five movies** to the user. 

Example recommendation logic for a user:

| User | Movie Title         | Rating |
|------|----------------------|--------|
| A    | Inception           | 4.5    |
| A    | Titanic             | 4.2    |
| A    | Avatar              | 2.8    |
| B    | Inception           | 4.6    |
| B    | The Matrix          | 3.9    |
| B    | Toy Story           | 4.7    |
| C    | Titanic             | 3.8    |
| C    | Avatar              | 4.3    |
| C    | Shrek               | 4.1    |

If User A and User B are identified as the two most similar users to User X, the recommended movies would be:
1. **Common Movies:** "Inception" (average rating: 4.55).
2. **Top-rated from Most Similar User:** "Toy Story" (4.7) from User B and "Titanic" (4.2) from User A.
3. If fewer than 5 movies are found, complete the list using other high-rated movies by the most similar users.

In [8]:
user_signatures = load_user_signatures_from_files(output_dir, 'diverse')

User signatures from loaded from /kaggle/working/user_signatures_diverse.json


In [31]:
# Define LSH Class
class LSH:
    def __init__(self, num_bands):
        """
        Initialize the LSH instance.
        Args:
            num_bands (int): Number of bands to split the signatures into.
        """
        self.num_bands = num_bands
        self.buckets = [{} for _ in range(num_bands)]  # List of dictionaries, one for each band

    async def add_user_signature(self, user_id, signature):
        rows_per_band = len(signature) // self.num_bands
        subvecs = [
            signature[i:i + rows_per_band]
            for i in range(0, len(signature), rows_per_band)
        ]
        for band_index, subvec in enumerate(subvecs):
            subvec_key = ','.join(map(str, subvec))
            if subvec_key not in self.buckets[band_index]:
                self.buckets[band_index][subvec_key] = []
            self.buckets[band_index][subvec_key].append(user_id)

    async def add_all_signatures(self, user_signatures):
        tasks = [
            self.add_user_signature(user_id, signature)
            for user_id, signature in user_signatures.items()
        ]
        await tqdm.gather(*tasks, desc="Adding user signatures", unit="user")

    async def find_similar_users(self, target_user_id, user_signatures, top_similar_users=1000):
        target_signature = user_signatures[target_user_id]
        rows_per_band = len(target_signature) // self.num_bands
        subvecs = [
            target_signature[i:i + rows_per_band]
            for i in range(0, len(target_signature), rows_per_band)
        ]

        similar_users = set()
        for band_index, subvec in enumerate(subvecs):
            subvec_key = ','.join(map(str, subvec))
            if subvec_key in self.buckets[band_index]:
                similar_users.update(self.buckets[band_index][subvec_key])

        similar_users.discard(target_user_id)

        # Rank similar users by Jaccard similarity
        def jaccard_similarity(sig1, sig2):
            return sum(1 for a, b in zip(sig1, sig2) if a == b) / len(sig1)

        ranked_users = sorted(
            similar_users,
            key=lambda user: jaccard_similarity(
                user_signatures[target_user_id], user_signatures[user]
            ),
            reverse=True,
        )

        return ranked_users[:top_similar_users]

async def recommend_movies(target_user_id, similar_users, ratings, top_k=5):
    """
    Generate movie recommendations based on similar users.

    Args:
        target_user_id: ID of the user for whom recommendations are being generated.
        similar_users: List of similar user IDs.
        ratings: DataFrame containing user ratings.
        top_k: Maximum number of movies to recommend.

    Returns:
        List of recommended movies.
    """
    # Ratings for the target user
    target_user_ratings = ratings[ratings["userId"] == target_user_id].set_index("movieId")["rating"]
    target_user_movies = set(target_user_ratings.index)

    # Ratings for similar users
    similar_users_ratings = ratings[ratings["userId"].isin(similar_users)]
    similar_users_ratings = similar_users_ratings.groupby("movieId")["rating"].mean()

    # Step 1: Find common movies
    common_movies = target_user_movies.intersection(similar_users_ratings.index)
    common_movies_avg = {
        movie: (target_user_ratings[movie] + similar_users_ratings[movie]) / 2
        for movie in common_movies
    }

    # Step 2: Find top-rated movies from similar users
    unique_movies = similar_users_ratings.drop(index=common_movies, errors='ignore')
    top_movies_from_similar_users = unique_movies.sort_values(ascending=False).head(top_k)

    # Step 3: Combine results
    recommendations = (
        sorted(common_movies_avg.items(), key=lambda x: x[1], reverse=True)  # Common movies (sorted by avg rating)
        + list(top_movies_from_similar_users.items())  # Top-rated movies from similar users
    )

    # Step 4: Limit to top_k recommendations
    recommendations = recommendations[:top_k]

    # Return only movie IDs
    return [movie for movie, _ in recommendations]

async def run_lsh(user_signatures, ratings, num_bands, target_user_id, top_k=5, top_similar_users=1000):
    """
    Run the LSH-based recommendation workflow.
    Args:
        user_signatures (dict): User signatures.
        ratings (pd.DataFrame): Ratings dataset.
        num_bands (int): Number of bands for LSH.
        user_id (int): Target user ID.
        top_k (int): Number of recommendations to return.
    Returns:
        list: List of recommended movie titles.
    """
    
    lsh = LSH(num_bands)

    print("Adding user signatures to LSH buckets...")
    await lsh.add_all_signatures(user_signatures)

    print("Finding similar users...")
    similar_users = await lsh.find_similar_users(target_user_id, user_signatures, top_similar_users=top_similar_users)
    # print(f"Similar Users for User {target_user_id}: {similar_users}")

    print("Generating recommendations...")
    recommended_movies = await recommend_movies(target_user_id, similar_users, ratings, top_k=top_k)
    print(f"Recommendations for User {target_user_id}: {recommended_movies}")

    return recommended_movies

To create LSH I used as a base this *source* and adapted to my user signatures.

`Source`: https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing/

In [42]:
def get_titles(recommended_movies, movie, rating, target_user_id):
    """
    Map a list of movie IDs to their corresponding titles and average ratings.

    Args:
        recommended_movies: List of movie IDs to map.
        movie: DataFrame containing movieId and title columns.
        rating: DataFrame containing userId, movieId, and rating columns.
        target_user_id: ID of the user for whom recommendations are being made.
    
    Returns:
        DataFrame with movie IDs, titles, and average ratings.
    """
    # Filter the rating dataframe for relevant movie IDs
    relevant_ratings = rating[rating['movieId'].isin(recommended_movies)]
    
    # Calculate average ratings for the recommended movies
    avg_ratings = relevant_ratings.groupby('movieId')['rating'].mean().reset_index()
    avg_ratings.columns = ['movieId', 'Average Rating']
    
    # Merge movie titles and average ratings
    recommended_movies_df = pd.DataFrame(recommended_movies, columns=["movieId"])
    mapped_movies = recommended_movies_df.merge(movie[['movieId', 'title']], on="movieId", how="left")
    mapped_movies = mapped_movies.merge(avg_ratings, on="movieId", how="left")
    
    # Rename columns for clarity
    mapped_movies = mapped_movies.rename(
        columns={"movieId": "Movie ID", "title": "Movie Title", "Average Rating": "Average Rating"}
    )
    
    # Select only the necessary columns
    mapped_movies = mapped_movies[["Movie ID", "Movie Title", "Average Rating"]]
    
    return mapped_movies

In [44]:
# Run LSH
target_user_id = 1
num_bands = 5
recommended_movies = await run_lsh(user_signatures, rating, num_bands, target_user_id)
recommendations = get_titles(recommended_movies, movie, rating, target_user_id)
recommendations

Adding user signatures to LSH buckets...


Adding user signatures: 100%|██████████| 138493/138493 [00:02<00:00, 47006.56user/s]


Finding similar users...
Generating recommendations...
Recommendations for User 1: [4993, 7153, 5952, 8507, 1196]


,Movie ID,Movie Title,Average Rating
0,4993,"Lord of the Rings: The Fellowship of the Ring,...",4.137925
1,7153,"Lord of the Rings: The Return of the King, The...",4.142382
2,5952,"Lord of the Rings: The Two Towers, The (2002)",4.107521
3,8507,Freaks (1932),3.801292
4,1196,Star Wars: Episode V - The Empire Strikes Back...,4.188202


In [45]:
target_user_id = 2
recommended_movies = await run_lsh(user_signatures, rating, num_bands, target_user_id)
recommendations = get_titles(recommended_movies, movie, rating, target_user_id)
recommendations

Adding user signatures to LSH buckets...


Adding user signatures: 100%|██████████| 138493/138493 [00:02<00:00, 48185.55user/s]


Finding similar users...
Generating recommendations...
Recommendations for User 2: [541, 260, 1196, 1214, 924]


,Movie ID,Movie Title,Average Rating
0,541,Blade Runner (1982),4.133706
1,260,Star Wars: Episode IV - A New Hope (1977),4.190672
2,1196,Star Wars: Episode V - The Empire Strikes Back...,4.188202
3,1214,Alien (1979),4.041784
4,924,2001: A Space Odyssey (1968),3.955748


## 4. Algorithmic Question

Two brilliant strategists, Arya and Mario, are about to play a game with a sequence of numbers. Arya, as player 1, begins the game, while Mario, player 2, plays 2nd. Their goal is clear: to collect the highest possible score by taking numbers from either end of the sequence, one at a time. They will play in perfect synchronicity, each seeking the advantage.

The sequence represented as an array of `nums,` is laid out in front of them. Arya will start by selecting either the number at the beginning (`nums[0]`) or the end (`nums[nums.length - 1]`) of the array, adding that value to her score. This value is then removed from the beginning or the end of `nums`. Then, it’s Mario’s turn to do the same with the remaining sequence. The game proceeds this way, with each player taking numbers from either end until no numbers are left to claim. The player with the highest score wins.

However, if they end in a tie, Arya, as the first to act, will claim victory by default.

Arya is now before you, asking for help to predict her chances. She wants to know, with her best possible choices, whether she can guarantee a win, assuming both players play with perfect skill.

- a) Help Arya by providing a pseudocode for finding an optimal playing strategy, that is, a strategy that maximizes her value. (Hint: Use recursion, assuming that both players play optimally).

- b) Write a Python program implementing her game strategy. Try different array lengths to test the algorithm.

- c) Is the algorithm efficient? Prove that it is polynomial and provide an asymptotic time complexity bound, or show that it requires exponential time.

- d) If the algorithm is exponential, explain how to make it polynomial and provide a pseudocode for it. Recompute the computational complexity of the updated algorithm.

- e) Implement the algorithm in Python. Compare your result values with the previous algorithm. Also compare the running times.

- f) Finally, consult LLM (ChatGPT, Claude AI, Gemini, Perplexity, etc.) to craft a third, optimized implementation and analyze its time complexity. Also, explain if the LLM is doing a good job and how you can evaluate whether the suggested solution works properly.

**Examples**

__Input 1__  
```
nums = [1, 5, 2]
```

__Output 1__  
```
false
```

__Explanation__: Arya’s optimal choices still lead her to a lower score than Mario’s, so she cannot guarantee victory.

__Input 2__  
```
nums = [1, 5, 233, 7]
```

__Output 2__  
```
true
```

__Explanation__: Arya, by playing perfectly, can ensure she ends up with the highest score.

---

**a) Help Arya by providing a pseudocode for finding an optimal playing strategy, that is, a strategy that maximizes her value. (Hint: Use recursion, assuming that both players play optimally).**

**BruteForceOptimalStrategy**

```plaintext
Function BruteForceOptimalStrategy(nums, start, end, is_aryas_turn)
Input:
    nums: List of integers representing the numbers in the game
    start: Current starting index of the array
    end: Current ending index of the array
    is_aryas_turn: Boolean indicating if it is Arya's turn (True for Arya, False for Mario)

Output:
    Integer: The maximum score difference Arya can achieve

    # Base case: if start exceeds end, no numbers are left to pick
    if start > end:
        return 0
    end if

    # If Arya's turn
    if is_aryas_turn:
        pick_start ← nums[start] + BruteForceOptimalStrategy(nums, start + 1, end, NOT is_aryas_turn)
        pick_end ← nums[end] + BruteForceOptimalStrategy(nums, start, end - 1, NOT is_aryas_turn)
        return Max(pick_start, pick_end)
    # If Mario's turn
    else:
        pick_start ← -nums[start] + BruteForceOptimalStrategy(nums, start + 1, end, NOT is_aryas_turn)
        pick_end ← -nums[end] + BruteForceOptimalStrategy(nums, start, end - 1, NOT is_aryas_turn)
        return Min(pick_start, pick_end)
    end if
    end function
```

**OptimalStrategy**

```plaintext
Function OptimalStrategy(nums)
Input:
    nums: List of integers representing the numbers in the game

Output:
    Boolean: True if Arya can guarantee a win, False otherwise

    score_diff ← BruteForceOptimalStrategy(nums, 0, Length(nums) - 1, True)
    return score_diff >= 0
    end function
```

**b) Write a Python program implementing her game strategy. Try different array lengths to test the algorithm.**

In [33]:
# exponential (brute force)
def brute_force_optimal_strategy(nums, start, end, is_aryas_turn):
    # Base case: if start exceeds end, no numbers are left to pick
    if start > end:
        return 0

    # If Arya's turn
    if is_aryas_turn:
        pick_start = nums[start] + brute_force_optimal_strategy(nums, start + 1, end, not is_aryas_turn)
        pick_end = nums[end] + brute_force_optimal_strategy(nums, start, end - 1, not is_aryas_turn)
        return max(pick_start, pick_end)
    # If Mario's turn
    else:  
        pick_start = -nums[start] + brute_force_optimal_strategy(nums, start + 1, end, not is_aryas_turn)
        pick_end = -nums[end] + brute_force_optimal_strategy(nums, start, end - 1, not is_aryas_turn)
        return min(pick_start, pick_end)
    
def optimal_strategy(nums):
    score_diff = brute_force_optimal_strategy(nums, 0, len(nums) - 1, True)
    # Arya wins if her score difference is non-negative (>= 0)
    return score_diff >= 0

nums1 = [1, 5, 2]
nums2 = [1, 5, 233, 7]
result = optimal_strategy(nums1)
print("Will Arya win?", result)

result = optimal_strategy(nums2)
print("Will Arya win?", result)

Will Arya win? False
Will Arya win? True


**c) Is the algorithm efficient? Prove that it is polynomial and provide an asymptotic time complexity bound, or show that it requires exponential time.**

The algorithm is not efficient cause explores all possible conminations. The **BruteForceOptimalStrategy** algorithm has an <u>asymptotic time complexity upper bound</u> of $O(2^n)$, indicating exponential growth that is based on two recursive calls at each step (pick `start` or `end`). 

**d) If the algorithm is exponential, explain how to make it polynomial and provide a pseudocode for it. Recompute the computational complexity of the updated algorithm.**

To make the algorithm polinomial it is possible to break the problems into $O(n^2)$ unique subproblems (one for each (`start`, `end`) pair), and each is solved once and stored in some temporary table as `memo`. Reducing recomputing (`start`, `end`) pairs memorizing them makes the <u>asymptotic time complexity upper bound</u> $O(n^2)$.

**MemoizedOptimalStrategy**

```plaintext
Function MemoizedOptimalStrategy(nums, start, end, memo, is_aryas_turn)
Input:
    nums: List of integers representing the numbers in the game
    start: Current starting index of the array
    end: Current ending index of the array
    memo: Dictionary to store the results of subproblems
    is_aryas_turn: Boolean indicating if it is Arya's turn (True for Arya, False for Mario)

Output:
    Integer: The maximum score difference Arya can achieve

    # Base case: if there is only one number left
    if start == end:
        return nums[start] if is_aryas_turn else -nums[start]
    end if

    # Check if the result is already in the memo
    if (start, end, is_aryas_turn) is in memo:
        return memo[(start, end, is_aryas_turn)]
    end if

    # If it is Arya's turn
    if is_aryas_turn:
        pick_start ← nums[start] + MemoizedOptimalStrategy(nums, start + 1, end, memo, NOT is_aryas_turn)
        pick_end ← nums[end] + MemoizedOptimalStrategy(nums, start, end - 1, memo, NOT is_aryas_turn)
        result ← Max(pick_start, pick_end)
    else:  # If it is Mario's turn
        pick_start ← -nums[start] + MemoizedOptimalStrategy(nums, start + 1, end, memo, NOT is_aryas_turn)
        pick_end ← -nums[end] + MemoizedOptimalStrategy(nums, start, end - 1, memo, NOT is_aryas_turn)
        result ← Min(pick_start, pick_end)
    end if
    
    # Store the result in the memo
    memo[(start, end, is_aryas_turn)] ← result
    
    return result
    end function
```

**OptimalStrategy**

```plaintext
Function OptimalStrategy(nums)
Input:
    nums: List of integers representing the numbers in the game

Output:
    Boolean: True if Arya can guarantee a win, False otherwise

    # Initialize the memoization dictionary
    memo ← {}
    
    # Calculate Arya's score difference starting with her turn
    score_diff ← MemoizedOptimalStrategy(nums, 0, Length(nums) - 1, memo, True)
    
    # Arya wins if her score difference is non-negative
    return score_diff >= 0
    end function

```

**e) Implement the algorithm in Python. Compare your result values with the previous algorithm. Also compare the running times.**

In [32]:
# polinomial (memorization of the steps)
def memoized_optimal_strategy(nums, start, end, memo, is_aryas_turn):
    # Base case: if there's only one number left
    if start == end:
        return nums[start] if is_aryas_turn else -nums[start]
    
    # Check if the result is already in the memo
    if (start, end, is_aryas_turn) in memo:
        return memo[(start, end, is_aryas_turn)]
    
    if is_aryas_turn:
        pick_start = nums[start] + memoized_optimal_strategy(nums, start + 1, end, memo, not is_aryas_turn)
        pick_end = nums[end] + memoized_optimal_strategy(nums, start, end - 1, memo, not is_aryas_turn)
        result = max(pick_start, pick_end)
    else:  # Mario's turn
        pick_start = -nums[start] + memoized_optimal_strategy(nums, start + 1, end, memo, not is_aryas_turn)
        pick_end = -nums[end] + memoized_optimal_strategy(nums, start, end - 1, memo, not is_aryas_turn)
        result = min(pick_start, pick_end)
    
    memo[(start, end, is_aryas_turn)] = result
    return result

def optimal_strategy(nums):
    memo = {}
    score_diff = memoized_optimal_strategy(nums, 0, len(nums) - 1, memo, True)
    return score_diff >= 0

nums1 = [1, 5, 2]
nums2 = [1, 5, 233, 7]
result = optimal_strategy(nums1)
print("Will Arya win?", result)

result = optimal_strategy(nums2)
print("Will Arya win?", result)

Will Arya win? False
Will Arya win? True


**f) Finally, consult LLM (ChatGPT, Claude AI, Gemini, Perplexity, etc.) to craft a third, optimized implementation and analyze its time complexity. Also, explain if the LLM is doing a good job and how you can evaluate whether the suggested solution works properly.**

In [34]:
def can_arya_win(nums):
    n = len(nums)
    # Precompute the cumulative sum for quick range sum queries
    total_sum = sum(nums)
    dp = [[0] * n for _ in range(n)]
    
    # Base case: when there's only one element
    for i in range(n):
        dp[i][i] = nums[i]
    
    # Fill the DP table for ranges of increasing length
    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            # Calculate the optimal score for Arya
            pick_left = nums[i] + (sum(nums[i+1:j+1]) - dp[i+1][j])
            pick_right = nums[j] + (sum(nums[i:j]) - dp[i][j-1])
            dp[i][j] = max(pick_left, pick_right)
    
    # Arya wins if her score is greater than half the total sum
    arya_score = dp[0][n-1]
    return arya_score > total_sum / 2

# Example usage
nums1 = [1, 5, 2]
nums2 = [1, 5, 233, 7]
print(can_arya_win(nums1))  # Output: True or False depending on input
print(can_arya_win(nums2))  # Output: True or False depending on input

False
True


**ChatGPT Responce:**

To solve this problem optimally with a polynomial-time algorithm, we can use **Dynamic Programming (DP)**. Here's how we can approach it:

**Problem Breakdown:**
1. The game is about maximizing your score by choosing numbers from either end of the sequence.
2. Both Arya and Mario play optimally, so each decision will impact the subsequent possible scores for both players.
3. Arya needs to determine if she can guarantee a win.

**Approach**:
We use a DP table where `dp[i][j]` represents the maximum score Arya can achieve when considering the subarray from index `i` to `j`. Here's the step-by-step explanation:

- **Base Case**: If `i == j` (only one number left), Arya takes that number:
  \[
  dp[i][i] = nums[i]
  \]

- **Transition**:
  When Arya picks from either end of the sequence:
  - If she picks `nums[i]`, Mario is left with the subarray `nums[i+1]...nums[j]`, and Arya's next optimal score is the remainder after Mario plays optimally.
  - If she picks `nums[j]`, the same reasoning applies for the subarray `nums[i]...nums[j-1]`.
  \[
  dp[i][j] = \max(nums[i] + \text{sum}[i+1][j] - dp[i+1][j], nums[j] + \text{sum}[i][j-1] - dp[i][j-1])
  \]
  Here, `sum[i][j]` is the sum of elements from `nums[i]` to `nums[j]` (used to compute Mario's optimal remaining score efficiently).

- **Goal**: Arya wins if her score (calculated in `dp[0][n-1]`) is strictly greater than half the total sum of the array.


**Explanation of Code:**
1. **`dp[i][j]`**:
   Tracks Arya’s best possible score for subarray `nums[i:j+1]`.
2. **Precomputing Sums**:
   The `sum(nums[i:j+1])` ensures Mario’s next score can be calculated quickly.
3. **Final Decision**:
   Arya wins if her computed score is greater than half of the total score.

**Complexity:**
- **Time Complexity**: \(O(n^2)\) due to the nested loops to compute the DP table.
- **Space Complexity**: \(O(n^2)\) for the DP table. Can be optimized to \(O(n)\) using rolling arrays.

This ensures an optimal polynomial-time solution to the problem.

To evaluate whether the suggested solution works properly I can trace it on one of the test cases:

**Rules Recap**
- **Arya (Player 1)**:
  Arya always starts. She aims to maximize her score and chooses optimally.
- **Mario (Player 2)**:
  Mario plays second and also chooses optimally to maximize his score (or minimize Arya’s advantage).

Both players can pick from either **end of the sequence**.

**Step-by-Step Breakdown for `nums = [1, 5, 233, 7]`**

**Trace Table (Arya's Decisions)**

|   | j=0            | j=1                      | j=2                        | j=3                       |
|---|----------------|--------------------------|----------------------------|---------------------------|
| i=0 | Pick 1         | Pick 5, remaining (0-0) | Pick 233, remaining (0-1)  | Pick 1, remaining (1-3)   |
| i=1 |                | Pick 5                  | Pick 233, remaining (1-1)  | Pick 7, remaining (1-2)   |
| i=2 |                |                          | Pick 233                   | Pick 233, remaining (3-3) |
| i=3 |                |                          |                            | Pick 7                    |


Going through each subarray and show Arya’s and Mario’s actions. Here's how the decisions are made:

**Subarrays of Length 1**
- **i=0, j=0**: Arya picks `1`.
- **i=1, j=1**: Arya picks `5`.
- **i=2, j=2**: Arya picks `233`.
- **i=3, j=3**: Arya picks `7`.

**Subarrays of Length 2**
- **i=0, j=1** (`[1, 5]`):
  - Arya can pick `1` (leaving `5` for Mario) or `5` (leaving `1` for Mario).
  - If Arya picks:
    - `1`: Mario picks `5`, total score for Arya is `1`.
    - `5`: Mario picks `1`, total score for Arya is `5`.
  - **Arya picks `5` for the best score.**

- **i=1, j=2** (`[5, 233]`):
  - Arya can pick `5` (leaving `233` for Mario) or `233` (leaving `5` for Mario).
  - If Arya picks:
    - `5`: Mario picks `233`, total score for Arya is `5`.
    - `233`: Mario picks `5`, total score for Arya is `233`.
  - **Arya picks `233` for the best score.**

- **i=2, j=3** (`[233, 7]`):
  - Arya can pick `233` (leaving `7` for Mario) or `7` (leaving `233` for Mario).
  - If Arya picks:
    - `233`: Mario picks `7`, total score for Arya is `233`.
    - `7`: Mario picks `233`, total score for Arya is `7`.
  - **Arya picks `233` for the best score.**

**Subarrays of Length 3**
- **i=0, j=2** (`[1, 5, 233]`):
  - Arya can pick:
    - `1`: Mario is left with `[5, 233]`. Mario picks `233` (leaving `5`), Arya gets `1 + 5 = 6`.
    - `233`: Mario is left with `[1, 5]`. Mario picks `5` (leaving `1`), Arya gets `233 + 1 = 234`.
  - **Arya picks `233` for the best score.**

- **i=1, j=3** (`[5, 233, 7]`):
  - Arya can pick:
    - `5`: Mario is left with `[233, 7]`. Mario picks `233` (leaving `7`), Arya gets `5 + 7 = 12`.
    - `7`: Mario is left with `[5, 233]`. Mario picks `233` (leaving `5`), Arya gets `7 + 5 = 12`.
  - **Arya picks `7` for the best score.**

**Full Array**
- **i=0, j=3** (`[1, 5, 233, 7]`):
  - Arya can pick:
    - `1`: Mario is left with `[5, 233, 7]`. Mario picks `233` (leaving `[5, 7]`), Arya can only secure `1 + 12 = 13`.
    - `7`: Mario is left with `[1, 5, 233]`. Mario picks `233` (leaving `[1, 5]`), Arya can only secure `7 + 6 = 13`.
  - **Arya picks `1` for the best outcome.**

**Final Score**

With perfect play, Arya can calculate her best outcome and whether she can win based on the initial choice.

**Key Takeaways for Decision Making:**
1. Arya always looks ahead to minimize Mario's best possible score.
2. Mario similarly plays optimally to force Arya into the worst-case scenario.
3. The DP table encodes all subarray decisions so Arya can ensure she makes the correct first move to maximize her final score.